In [9]:
import cv2
import mediapipe as mp
import matplotlib.pyplot as plt
import numpy as np

In [10]:
mp_pose = mp.solutions.pose
 
pose_image = mp_pose.Pose(static_image_mode=True, 
                          min_detection_confidence=0.5)
 
pose_video = mp_pose.Pose(static_image_mode=False, 
                          min_detection_confidence=0.7,
                          min_tracking_confidence=0.7)
 
mp_drawing = mp.solutions.drawing_utils

pose = mp_pose.Pose()

In [11]:
def detectPose(image_pose, pose, draw=False):
    original_image = image_pose.copy()
    image_in_RGB = cv2.cvtColor(image_pose, cv2.COLOR_BGR2RGB)
    resultant = pose.process(image_in_RGB)
    if resultant.pose_landmarks and draw:    
        mp_drawing.draw_landmarks(image=original_image, 
                                  landmark_list=resultant.pose_landmarks,
                                  connections=mp_pose.POSE_CONNECTIONS,
                                  landmark_drawing_spec=mp_drawing.DrawingSpec(color=(255,255,255),
                                                                               thickness=2, circle_radius=2),
                                  connection_drawing_spec=mp_drawing.DrawingSpec(color=(49,125,237),
                                                                               thickness=2, circle_radius=2))
            
    return original_image, resultant


In [12]:
# PREPARING IMAGE
image = cv2.imread("test_slika.jpg")
slika, res = detectPose(image, pose, True)
slika = cv2.cvtColor(slika, cv2.COLOR_BGR2RGB)


In [15]:
# PREPARING DATA - DICTIONARY WITH POINTS ON ALL AXIS

points = []
for data_point in res.pose_landmarks.landmark:
    points.append({
                    'X': data_point.x,
                    'Y': data_point.y,
                    'Z': data_point.z,
                    'Visibility': data_point.visibility})

print(points[11])
print(points[24])

spineTopX1 = points[11]["X"]
spineTopY1 = points[11]["Y"]
spineTopZ1 = points[11]["Z"]

spineTopX2 = points[12]["X"]
spineTopY2 = points[12]["Y"]
spineTopZ2 = points[12]["Z"]

spineBotX1 = points[23]["X"]
spineBotY1 = points[23]["Y"]
spineBotZ1 = points[23]["Z"]

spineBotX2 = points[24]["X"]
spineBotY2 = points[24]["Y"]
spineBotZ2 = points[24]["Z"]

vec1 = np.array([spineTopX1, spineTopY1, spineTopZ1])
vec2 = np.array([spineTopX2, spineTopY2, spineTopZ2])

vec3 = np.array([spineBotX1, spineBotY1, spineBotZ1])
vec4 = np.array([spineBotX2, spineBotY2, spineBotZ2])

spineTop = (vec1 + vec2) / 2
spineBot = (vec3 + vec4) / 2

print(spineTop)

{'X': 0.6119961142539978, 'Y': 0.42453041672706604, 'Z': 0.09047698974609375, 'Visibility': 0.9999719858169556}
{'X': 0.47624024748802185, 'Y': 0.6151109337806702, 'Z': 0.012838715687394142, 'Visibility': 0.9995971322059631}
[0.532847   0.42221259 0.12269718]


In [6]:
import mediapipe

mediapipe

<module 'mediapipe' from 'c:\\Users\\Gasper Krevs\\AppData\\Local\\Programs\\Python\\Python310\\lib\\site-packages\\mediapipe\\__init__.py'>